In [107]:
import chess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle


%cd /home/assaf/jupyter/chess
###!rm -fr all_states_list.pkl

def get_np_board(board_):
    matrix = np.zeros((8, 8), dtype=np.int8)
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 2,
        chess.BISHOP: 3,
        chess.ROOK: 4,
        chess.QUEEN: 5,
        chess.KING: 6,
    }
    for square, piece in board.piece_map().items():
        row = 7 - chess.square_rank(square)
        col = chess.square_file(square)
        value = piece_values[piece.piece_type]
        matrix[row, col] = value if piece.color == chess.WHITE else -value    
    return matrix

states = []
print()
df = pd.read_csv("games.csv")

try:
    with open("all_states_list.pkl", "rb") as f:
        states = pickle.load(f)
        print('loaded states from disk successfully')

except Exception:
    print('cannot load states from disk. calculating')
    for i in range(len(df)):
        board = chess.Board()
        # if i>0: break
        thisgame = df.loc[i]
        game_san = thisgame.moves.split()
        for j, step_san in enumerate(game_san):
            move = board.parse_san(step_san)  # Converts SAN to Move
            board.push(move)  # Make the move on the board
            # print(board)
            # print(str(board).replace(" ", "").replace("\n", "/"))
            # if j<5:
            #     continue
            state = []
            state.append(j)
            state.append(thisgame['id'])
            state.append(thisgame['turns']) 
            state.append(thisgame['victory_status'])
            state.append(thisgame['winner'])
            state.append(thisgame['white_rating'])
            state.append(thisgame['black_rating'])
            state.append(str(board).replace(" ", "").replace("\n", "/"))
            state.append(str(board).replace(" ", "").replace("\n", ""))
            state.append(get_np_board(board))
            states.append(state)
            # print(str(board).replace(" ", "").replace("\n", ""))
        print (i, len(states))
    with open("all_states_list.pkl", "wb") as f:
        pickle.dump(states, f)
states_df = pd.DataFrame(data, columns=["turn", "id", "turns", 'victory_status', 'winner', 'white_rating', 'black_rating', '1d_board', '1d_board_', 'matrix'])        
states_df.head(10)

/home/assaf/jupyter/chess

loaded states from disk successfully


,turn,id,turns,victory_status,winner,white_rating,black_rating,1d_board,1d_board_,matrix
0,0,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pppppppp/......../......../...P..../....,rnbqkbnrpppppppp...................P.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
1,1,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/ppp.pppp/......../...p..../...P..../....,rnbqkbnrppp.pppp...........p.......P.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
2,2,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/ppp.pppp/......../...p..../..PP..../....,rnbqkbnrppp.pppp...........p......PP.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1..."
3,3,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pp..pppp/..p...../...p..../..PP..../....,rnbqkbnrpp..pppp..p........p......PP.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, 0,..."
4,4,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pp..pppp/..p...../...P..../...P..../....,rnbqkbnrpp..pppp..p........P.......P.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, 0,..."
5,5,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pp...ppp/..p.p.../...P..../...P..../....,rnbqkbnrpp...ppp..p.p......P.......P.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, 0,..."
6,6,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pp...ppp/..p.P.../......../...P..../....,rnbqkbnrpp...ppp..p.P..............P.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, 0,..."
7,7,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pp....pp/..p.p.../......../...P..../....,rnbqkbnrpp....pp..p.p..............P.............,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, 0,..."
8,8,TZJHLljE,13,outoftime,white,1500,1191,rnbqkbnr/pp....pp/..p.p.../......../...P..../....,rnbqkbnrpp....pp..p.p..............P.........N...,"[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, 0,..."
9,9,TZJHLljE,13,outoftime,white,1500,1191,rnbqk.nr/pp....pp/..p.p.../......../.b.P..../....,rnbqk.nrpp....pp..p.p............b.P.........N...,"[[-4, -2, -3, -5, -6, 0, -2, -4], [-1, -1, 0, ..."


In [102]:
state_str = [row[8] for row in states]

from collections import Counter

counts = Counter(state_str)
stats = counts.most_common()
sum = [row[1] for row in stats ]
n=2
print(f'number of states that have more than {n} occuance {np.mean(np.array(sum)>n):.01%}')
# stats[::-1]

number of states that have more than 2 occuance 1.5%
